<a href="https://colab.research.google.com/github/liuxiu842/ML/blob/master/apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install efficient_apriori
import pandas as pd
import time
pd.options.display.max_columns=100

# 数据加载
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/L3/BreadBasket_DMS.csv')
print(data)
# 统一小写
data['Item'] = data['Item'].str.lower()
print(data)
# 去掉none项
data = data.drop(data[data.Item == 'none'].index)
print(data)


In [0]:
# 采用efficient_apriori工具包
def rule1():
	from efficient_apriori import apriori
	start = time.time()
	# 得到一维数组orders_series，并且将Transaction作为index, value为Item取值
	orders_series = data.set_index('Transaction')['Item']
	# 将数据集进行格式转换
	transactions = []
	temp_index = 0
	for i, v in orders_series.items():
		if i != temp_index:
			temp_set = set()
			temp_index = i
			temp_set.add(v)
			transactions.append(temp_set)
		else:
			temp_set.add(v)
	
	# 挖掘频繁项集和频繁规则
	itemsets, rules = apriori(transactions, min_support=0.02,  min_confidence=0.5)
	print('频繁项集：', itemsets)
	print('关联规则：', rules)
	end = time.time()
	print("用时：", end-start)

rule1()


In [0]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
# 采用mlxtend.frequent_patterns工具包
def rule2():
	from mlxtend.frequent_patterns import apriori
	from mlxtend.frequent_patterns import association_rules
	pd.options.display.max_columns=100
	start = time.time()
	hot_encoded_df=data.groupby(['Transaction','Item'])['Item'].count().unstack().reset_index().fillna(0).set_index('Transaction')
	print(hot_encoded_df)
	hot_encoded_df = hot_encoded_df.applymap(encode_units)
	print(hot_encoded_df)
	frequent_itemsets = apriori(hot_encoded_df, min_support=0.02, use_colnames=True)
	rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.5)
	print("频繁项集：", frequent_itemsets)
	print("关联规则：", rules[ (rules['lift'] >= 1) & (rules['confidence'] >= 0.5) ])
	#print(rules['confidence'])
	end = time.time()
	print("用时：", end-start)


print('-'*100)
rule2()


In [0]:
#!pip install efficient_apriori
import pandas as pd
import time
pd.options.display.max_columns=100

# 数据加载
market = pd.read_csv('/content/drive/My Drive/Colab Notebooks/L3/Market_Basket_Optimisation.csv', header=None)
market.head()


In [0]:
for i in market.columns:
    market[i] = market[i].str.strip()
market.head()

In [0]:
market_hot_encode=pd.DataFrame(columns=["items"])
for i in market.index:
    item=""
    for j in market.loc[i].unique():
        item+=str(j)+"|"
    market_hot_encode.loc[i,"items"]=item

market_hot_encode.head()
market_hot_encode.info()   

In [0]:
# one-hot编码,并删除nan列
market_hot_encode=market_hot_encode["items"].str.get_dummies(sep="|").drop("nan",axis=1)

market_hot_encode.head()


In [0]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

itemsets = apriori(market_hot_encode,use_colnames=True, min_support=0.025)
print(itemsets)
itemsets = itemsets.sort_values(by="support" , ascending=False) 
print('-'*20, '频繁项集', '-'*20)
itemsets

In [0]:
# 根据频繁项集计算关联规则，设置最小提升度为1.5
rules =  association_rules(itemsets, metric='lift', min_threshold=1.5)
# 按照提升度从大到小进行排序
rules = rules.sort_values(by="lift" , ascending=False)
rules

In [0]:
from efficient_apriori import apriori
itemsets, rules = apriori(market, min_support=0.02,  min_confidence=0.5)
print('频繁项集：', itemsets)
print('关联规则：', rules)